Домашнее задание:
1. Задание 1: Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.
2. Задание 2: Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.
3. Задание 3: Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).
4. Задание 4: Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).
5. Задание 5: Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.
6. Задание 6: Могла ли модель переобучиться? Почему?
7. Задание 7 (опциональное): Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [4]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

%matplotlib inline

Задание 1: Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [5]:
def calc_logloss(y_true, y_pred):
    k = 1e-5
    y_pred = y_pred.copy()
    y_pred = np.clip(y_pred, a_min=k, a_max=1-k)
    err = - np.mean(y_true * np.log(y_pred) + (1.0 - y_true) * np.log(1.0 - y_pred))
    return err

Задание 2: Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [6]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [7]:
X

array([[1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00,
        1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 3.00e+00, 0.00e+00,
        5.00e+00, 1.00e+01, 1.00e+00, 2.00e+00],
       [5.00e+02, 7.00e+02, 7.50e+02, 6.00e+02, 1.45e+03, 8.00e+02,
        1.50e+03, 2.00e+03, 4.50e+02, 1.00e+03],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 2.00e+00, 1.00e+00,
        3.00e+00, 3.00e+00, 1.00e+00, 2.00e+00]])

In [8]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [9]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [10]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [11]:
X = X_st
X

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [12]:
def sigmoid(z):
    rec = 1 / (1 + np.exp(-z))
    return rec

In [13]:
def eval_model(X, y, iteration, alpha=1e-4, k=1e-5):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    err = []
    w = []
    
    for n_iter in range (1, iteration+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err_new = calc_logloss(y, y_pred)
        W = W - alpha*(1/n * np.dot((y_pred - y), X.T))
        #W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        err.append(err_new)
        w.append(W)
        
        if n_iter > 2 and np.abs(err_new - err[-2]) < k:
            print(n_iter, W, err_new)
            break
    err = np.array(err)
    return err, w

In [14]:
learning_model = [0.1, 0.01, 0.001, 0.0001]

for alpha1 in learning_model:
    W = eval_model(X, y, iteration=1000000, alpha=alpha1)

5853 [-8.53579974 -1.24563049 -1.51151549  7.33020294] 0.28384397239036285
6823 [-2.01991797 -0.97320565  0.83993406  2.78408516] 0.43336113303724266
3017 [ 0.06146141 -0.68603187  0.85260956  1.21305757] 0.5558187108126926
6890 [ 0.30484831 -0.64299824  0.65968825  1.27144238] 0.6088502480761402


In [15]:
for alpha in [0.1, 0.01, 0.001, 0.0001]:
    err_1, w_1 = eval_model(X=X, y=y, iteration = 1000000, alpha=alpha)

5853 [-8.53579974 -1.24563049 -1.51151549  7.33020294] 0.28384397239036285
6823 [-2.01991797 -0.97320565  0.83993406  2.78408516] 0.43336113303724266
3017 [ 0.06146141 -0.68603187  0.85260956  1.21305757] 0.5558187108126926
6890 [ 0.30484831 -0.64299824  0.65968825  1.27144238] 0.6088502480761402


3. Задание 3: Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [16]:
def calc_pred_proba(W, X):
    y_pred = 1 /(1+ np.exp(-np.dot(W, X)))
    return y_pred

In [17]:
calc_pred_proba(w_1[-1], X)

array([0.57127834, 0.63625882, 0.77827302, 0.60422858, 0.82706424,
       0.79219513, 0.83457872, 0.2857075 , 0.55454336, 0.83142332])

Задание 4: Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [18]:
def calc_pred(W, X, threshold=0.5):
    y_pred = calc_pred_proba(W, X)
    y_pred = np.where(y_pred > threshold, 1, 0)
    return y_pred

In [27]:
y_pred = calc_pred(w_1[-1], X)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1])

Задание 5: Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [25]:
def accuracy(y, y_pred):
    value = np.arange(10)
    for i in range(len(y_pred)):
        if (y_pred[i] == y[i]): 
            value[i] = 1
        elif (y_pred[i] != y[i]): 
            value[i] = 0
    accuracy = sum(value)/len(y_pred)
    return accuracy

In [26]:
accuracy = accuracy(y, y_pred)
accuracy

NameError: name 'y_pred' is not defined

In [37]:
def confusion_matrix(y, y_pred):
    conf_matrix = np.zeros(shape=(2,2))
    # находим значение tp
    conf_matrix[0,0] = np.sum(np.where(y==1,y==y_pred,False))
    # находим значение tn
    conf_matrix[1,1] = np.sum(np.where(y==0,y==y_pred,False))
    # находим значение fn
    conf_matrix[1,0] = np.sum(np.where(y==1,y!=y_pred,False))
    # находим значение fp
    conf_matrix[0,1] = np.sum(np.where(y==0,y!=y_pred,False))
    
    return conf_matrix.astype(int)

In [38]:
conf_matrix = confusion_matrix(y, y_pred)
conf_matrix

array([[5, 4],
       [0, 1]])

In [49]:
def accuracy_score (y, y_pred):
    tp, tn, fn, fp = conf_matrix.flatten()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return accuracy

In [58]:
accuracy = accuracy_score(y, y_pred)
accuracy

0.9

In [56]:
def preсision_score (y, y_pred):            
    tp, tn, fn, fp = conf_matrix.flatten()
    preсision = tp /(tp + fp)
    return preсision

In [66]:
preсision = preсision_score (y, y_pred)
preсision

0.8333333333333334

In [62]:
def recall_score (y, y_pred):
    tp, tn, fn, fp = conf_matrix.flatten()
    recall = tp / (tp + fn)
    return recall

In [63]:
recall = recall_score(y, y_pred)
recall

1.0

In [67]:
def f1_score (y, y_pred):            
    tp, tn, fn, fp = conf_matrix.flatten()
    f1 = 2 * preсision * recall / (preсision + recall)
    return f1

In [68]:
f1 = f1_score (y, y_pred)
f1

0.9090909090909091